In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta
from landsatxplore.api import API
from crawlers.csv_reader import csv_crawler
from crawlers.web_crawler_earthexplorer import get_dates

In [2]:
global data
global data_grouped

In [3]:
def scene_finder(api=None, sat_choice=None, date=None, coordinates=None):
    scenes = None
    landsat_collection = "landsat_8_c1"
    sentinel_collection = "sentinel_2a"

    [xmin, ymin, xmax, ymax] = coordinates
    # print(xmin, ymin, xmax, ymax)
    if sat_choice == "1":
        scenes = api.search(
            dataset=landsat_collection,
            bbox=(xmin,
                  ymin,
                  xmax,
                  ymax),
            start_date=date[0],
            end_date=date[1],
        )
    elif sat_choice == "2":
        scenes = api.search(
            dataset=sentinel_collection,
            bbox=(xmin,
                  ymin,
                  xmax,
                  ymax),
            start_date=date[0],
            end_date=date[1],
        )

    print(f"{len(scenes)} scenes found.")
    return scenes

In [34]:
def scene_downloader(scene_list=None):
    for scene in scene_list:
        print(scene)
        os.system(
            f"landsatxplore download {scene} --output /home/chiko/Storage/Projects/Raster_Image_Calculator/Images/Request.5/Start_date --username {username} --password {password}")

In [23]:
def download_selector(scenes=None, test=False):
    data = pd.DataFrame()
    columns = ['entity_id', 'wrs_path', 'wrs_row', 'cloud_cover', 'start_time']
    for scene in scenes:
        temp = {column: scene[column] for column in columns}
        data = data.append(temp, ignore_index=True)
#         print(temp)
    data_grouped = data.groupby(['wrs_path', 'wrs_row'])
    down_list = []
    for group in data_grouped.groups:
        grp = data_grouped.get_group(group)
        grp.sort_values(['cloud_cover'], inplace=True)
        print(grp.loc[grp.index[0], ['wrs_path', 'wrs_row']])
        print(grp)
        print(f"min = {grp.iloc[0]['cloud_cover']}")
        print(f"max = {grp.iloc[-1]['cloud_cover']}")
        print("==========================================================================================")
        down_list.append(grp.iloc[0]['entity_id'])
    
    if test:
        return down_list, data
    return down_list

In [6]:
username = "vineet"
password = "EROS#3mobiscuit"

sat_choice = input("Choose satellite:\n1. Landsat\n2.Sentinel\n")

In [35]:
# def downloader():
api = API(username, password)
choice = "1"

date_list = get_dates(sat_choice=choice, test=True)

co_ord = csv_crawler(block="Mandla", clipper=True)
# print(co_ord)
for (id, date) in enumerate(date_list):
    scenes = scene_finder(api, sat_choice, date, coordinates=co_ord)
    scene_list, scene_data = download_selector(scenes, test=True)
    scene_downloader(scene_list)
api.logout()

13 scenes found.
wrs_path    143.0
wrs_row      44.0
Name: 3, dtype: object
    cloud_cover              entity_id                 start_time  wrs_path  \
3          0.00  LC81430442021054LGN00 2021-02-23 05:01:40.153123     143.0   
6          0.00  LC81430442021038LGN00 2021-02-07 05:01:45.033803     143.0   
8          0.00  LC81430442021022LGN00 2021-01-22 05:01:46.878754     143.0   
0         21.00  LC81430442021070LGN00 2021-03-11 05:01:31.935023     143.0   
11        78.07  LC81430442021006LGN00 2021-01-06 05:01:53.102718     143.0   

    wrs_row  
3      44.0  
6      44.0  
8      44.0  
0      44.0  
11     44.0  
min = 0.0
max = 78.07
wrs_path    143.0
wrs_row      45.0
Name: 4, dtype: object
    cloud_cover              entity_id                 start_time  wrs_path  \
4          0.00  LC81430452021054LGN00 2021-02-23 05:02:04.044162     143.0   
7          0.00  LC81430452021038LGN00 2021-02-07 05:02:08.933314     143.0   
9          0.00  LC81430452021022LGN00 2021-01-

In [32]:
scene_list

['LC81430442021054LGN00', 'LC81430452021054LGN00', 'LC81440442021061LGN00']

In [40]:
scene_data

,cloud_cover,entity_id,start_time,wrs_path,wrs_row
0,21.00,LC81430442021070LGN00,2021-03-11 05:01:31.935023,143.0,44.0
1,33.33,LC81430452021070LGN00,2021-03-11 05:01:55.826062,143.0,45.0
2,0.00,LC81440442021061LGN00,2021-03-02 05:07:47.793362,144.0,44.0
3,0.00,LC81430442021054LGN00,2021-02-23 05:01:40.153123,143.0,44.0
4,0.00,LC81430452021054LGN00,2021-02-23 05:02:04.044162,143.0,45.0
5,0.39,LC81440442021045LGN00,2021-02-14 05:07:54.101590,144.0,44.0
6,0.00,LC81430442021038LGN00,2021-02-07 05:01:45.033803,143.0,44.0
7,0.00,LC81430452021038LGN00,2021-02-07 05:02:08.933314,143.0,45.0
8,0.00,LC81430442021022LGN00,2021-01-22 05:01:46.878754,143.0,44.0
9,0.00,LC81430452021022LGN00,2021-01-22 05:02:10.782501,143.0,45.0


In [27]:
data_grouped = scene_data.groupby(['wrs_path', 'wrs_row'])
# data_grouped.groups

{(143.0, 44.0): [0, 3, 6, 8, 11], (143.0, 45.0): [1, 4, 7, 9, 12], (144.0, 44.0): [2, 5, 10]}

In [39]:
count = 0
for group in data_grouped.groups:
    if count == 1:
        break
    count += 1
    grp = data_grouped.get_group(group)
    grp.sort_values(['cloud_cover'], inplace=True)
    print(grp.loc[grp.index[0], ['wrs_path', 'wrs_row']])
    print(grp)
    print(f"min = {grp.iloc[0]['cloud_cover']}")
    print(f"max = {grp.iloc[-1]['cloud_cover']}")
    print("==========================================================================================")

wrs_path    143.0
wrs_row      44.0
Name: 3, dtype: object
    cloud_cover              entity_id                 start_time  wrs_path  \
3          0.00  LC81430442021054LGN00 2021-02-23 05:01:40.153123     143.0   
6          0.00  LC81430442021038LGN00 2021-02-07 05:01:45.033803     143.0   
8          0.00  LC81430442021022LGN00 2021-01-22 05:01:46.878754     143.0   
0         21.00  LC81430442021070LGN00 2021-03-11 05:01:31.935023     143.0   
11        78.07  LC81430442021006LGN00 2021-01-06 05:01:53.102718     143.0   

    wrs_row  
3      44.0  
6      44.0  
8      44.0  
0      44.0  
11     44.0  
min = 0.0
max = 78.07
<ipython-input-39-7f6b3904f54c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grp.sort_values(['cloud_cover'], inplace=True)
